In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [3]:
# ✅ Imports
import os
import cv2
import numpy as np
import gradio as gr
from google.colab import drive
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ Constants
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 10
DATASET_PATH = '/content/drive/My Drive/Brain Tumor Segmentation/Training'  # Make sure the path is correct

CATEGORIES = ['glioma', 'meningioma', 'notumor', 'pituitary']
data = []
labels = []

# ✅ Load and resize images
for category in CATEGORIES:
    folder_path = os.path.join(DATASET_PATH, category)
    print(f"Loading {category}...")
    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img)
        try:
            image = cv2.imread(img_path)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            data.append(image)
            labels.append(category)
        except Exception as e:
            print(f"Could not process image {img_path}: {e}")
            continue

# ✅ Preprocess data
data = np.array(data) / 255.0
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# ✅ Define model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(CATEGORIES), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# ✅ Train model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), epochs=EPOCHS, verbose=1)

# ✅ Prediction function
def predict_image(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    label = lb.inverse_transform(prediction)
    return label[0]

# ✅ Gradio interface
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type='pil'),
    outputs=gr.Label(num_top_classes=4),
    title="Brain Tumor Classifier",
    description="Upload an image to classify a brain tumor as glioma, meningioma, notumor, or pituitary."
)

interface.launch()


Mounted at /content/drive
Loading glioma...
Loading meningioma...
Loading notumor...
Loading pituitary...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 17s 33ms/step - accuracy: 0.6606 - loss: 0.8657 - val_accuracy: 0.2616 - val_loss: 8.8371
Epoch 2/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.7574 - loss: 0.6251 - val_accuracy: 0.6465 - val_loss: 0.9882
Epoch 3/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8023 - loss: 0.5190 - val_accuracy: 0.7900 - val_loss: 0.5633
Epoch 4/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8388 - loss: 0.4473 - val_accuracy: 0.5048 - val_loss: 1.4176
Epoch 5/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8261 - loss: 0.4620 - val_accuracy: 0.7830 - val_loss: 0.6010
Epoch 6/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8709 - loss: 0.3532 - val_accuracy: 0.6194 - val_loss: 2.0234
Epoch 7/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8757 - loss: 0.3294 - val_accuracy: 0.8399 - val_loss: 0.4219
Epoch 8/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8774 - loss: 0.3343 - val_ac